In [4]:
import numpy as np
import struct
def load_images(file_name):
    ##   在读取或写入一个文件之前，你必须使用 Python 内置open()函数来打开它。##
    ##   这里rb表示只能以二进制读取的方式打开一个文件                      ##
    binfile = open(file_name, 'rb') 
    ##   从一个打开的文件读取数据
    buffers = binfile.read()
    ##   读取image文件前4个整型数字
    magic,num,rows,cols = struct.unpack_from('>IIII',buffers, 0)
    ##   整个images数据大小为60000*28*28
    bits = num * rows * cols
    ##   读取images数据
    images = struct.unpack_from('>' + str(bits) + 'B', buffers, struct.calcsize('>IIII'))
    ##   关闭文件
    binfile.close()
    ##   转换为[60000,784]型数组
    images = np.reshape(images, [num, rows * cols])
    return images

def load_labels(file_name):
    ##   打开文件
    binfile = open(file_name, 'rb')
    ##   从一个打开的文件读取数据    
    buffers = binfile.read()
    ##   读取label文件前2个整形数字，label的长度为num
    magic,num = struct.unpack_from('>II', buffers, 0) 
    ##   读取labels数据
    labels = struct.unpack_from('>' + str(num) + "B", buffers, struct.calcsize('>II'))
    ##   关闭文件
    binfile.close()
    ##   转换为一维数组
    labels = np.reshape(labels, [num])
    return labels

In [5]:
filename_train_images = "MNIST/train-images-idx3-ubyte"
filename_train_labels = 'MNIST/train-labels-idx1-ubyte'
##filename_test_images = 'MNIST/t10k-images.idx3-ubyte'
##filename_test_labels = 'MNIST/t10k-labels.idx1-ubyte'
## train_images=load_images(filename_train_images)
## train_labels=load_labels(filename_train_labels)
##test_images=load_images(filename_test_images)
##test_labels=load_labels(filename_test_labels)

In [1]:
## 打印图片
import matplotlib
matplotlib.use('TkAgg') ##macOS需要换一下后端才能显示图片。
from matplotlib import pyplot as plt

fig=plt.figure(figsize=(8,8))
fig.subplots_adjust(left=0,right=1,bottom=0,top=1,hspace=0.05,wspace=0.05)
for i in range(30):
    images = np.reshape(train_images[i], [28,28])
    ax=fig.add_subplot(6,5,i+1,xticks=[],yticks=[])
    ax.imshow(images,cmap=plt.cm.binary,interpolation='nearest')
    ax.text(0,7,str(train_labels[i]))
plt.show()

NameError: name 'np' is not defined

# KNN算法实现

# 读取数据 image label

In [3]:
from numpy import *
import operator
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from os import listdir
from mpl_toolkits.mplot3d import Axes3D
import struct

#读取图片
def read_image(file_name):
    #先用二进制方式把文件都读进来
    file_handle=open(file_name,"rb")  #以二进制打开文档
    file_content=file_handle.read()   #读取到缓冲区中

    offset=0
    head = struct.unpack_from('>IIII', file_content, offset)  # 取前4个整数，返回一个元组
    offset += struct.calcsize('>IIII')
    imgNum = head[1]  #图片数
    rows = head[2]   #宽度
    cols = head[3]  #高度
    # print(imgNum)
    # print(rows)
    # print(cols)

    #测试读取一个图片是否读取成功
    #im = struct.unpack_from('>784B', file_content, offset)
    #offset += struct.calcsize('>784B')

    images=np.empty((imgNum , 784))#empty，是它所常见的数组内的所有元素均为空，没有实际意义，它是创建数组最快的方法
    image_size=rows*cols#单个图片的大小
    fmt='>' + str(image_size) + 'B'#单个图片的format

    for i in range(imgNum):
        images[i] = np.array(struct.unpack_from(fmt, file_content, offset))
        # images[i] = np.array(struct.unpack_from(fmt, file_content, offset)).reshape((rows, cols))
        offset += struct.calcsize(fmt)

    return images

    '''bits = imgNum * rows * cols  # data一共有60000*28*28个像素值
    bitsString = '>' + str(bits) + 'B'  # fmt格式：'>47040000B'
    imgs = struct.unpack_from(bitsString, file_content, offset)  # 取data数据，返回一个元组
    imgs_array=np.array(imgs).reshape((imgNum,rows*cols))     #最后将读取的数据reshape成 【图片数，图片像素】二维数组
    return imgs_array'''

def read_label(file_name):
    file_handle = open(file_name, "rb")  # 以二进制打开文档
    file_content = file_handle.read()  # 读取到缓冲区中

    head = struct.unpack_from('>II', file_content, 0)  # 取前2个整数，返回一个元组
    offset = struct.calcsize('>II')

    labelNum = head[1]  # label数
    # print(labelNum)
    bitsString = '>' + str(labelNum) + 'B'  # fmt格式：'>47040000B'
    label = struct.unpack_from(bitsString, file_content, offset)  # 取data数据，返回一个元组
    return np.array(label)

# KNN Train

In [4]:
# KNN算法
def KNN(test_data, images, labels, k):
    # images.shape[0]表示的是读取矩阵第一维度的长度，代表行数60k
    dataSetSize = images.shape[0]  # 60k
    # tile函数在行上重复dataSetSize次，在列上重复1次
    # 将test_data乘60k。再reshape成784行*60000列
    distance1 = tile(test_data, (dataSetSize)).reshape((dataSetSize,784))-images
    # 每个元素平方
    distance2 = distance1**2
    # 矩阵每行相加
    distance3 = distance2.sum(axis=1) # 合并成一行
    # 开方
    distances4 = distance3**0.5
    # 欧氏距离计算结束
    # 返回从小到大排序的索引
    sortedDistIndicies = distances4.argsort()
    classCount=np.zeros((10), np.int32) #10是代表10个类别
    for i in range(k): # 统计排在前k名的预测结果
        ## 找到图片对应的数据
        voteIlabel = labels[sortedDistIndicies[i]] 
        ## 把这个Count + 1
        classCount[voteIlabel] += 1

    return np.argmax(classCount)

# KNN Test

In [15]:
def KNN_TEST(train_x,test_x,train_y,test_y,k):    

    testRatio = 1  # 取数据集的前0.1为测试数据,这个参数比重可以改变
    train_row = train_x.shape[0]  # 数据集的行数，即数据集的总的样本数
    test_row=test_x.shape[0]
    print('测试图片数量:',test_row)
    testNum = int(test_row * testRatio)
    errorCount = 0  # 判断错误的个数
    error = []
    for i in range(testNum):
        result = KNN(test_x[i], train_x, train_y, k)
    errorCount = 0  # 判断错误的个数
    for i in range(testNum):
        result = KNN(test_x[i], train_x, train_y, k)
        # print('返回的结果是: %s, 真实结果是: %s' % (result, test_y[i]))
        if result != test_y[i]:
            errorCount += 1.0 # 如果mnist验证集的标签和本身标签不一样，则出错
            print('返回的结果是: %s, 真实结果是: %s' % (result, test_y[i]))
            error.append(i)
        # print(i)
    
    error_rate = errorCount / float(testNum)  # 计算出错率
    acc = 1.0 - error_rate
    print("\n正确预测率: %f" % (acc))
    print("\n错误预测率: %f" % (error_rate))
    print("\n错误预测的图片数量: %d" % errorCount)
    return error

# 传入数据

In [16]:
train_image = "MNIST/train-images-idx3-ubyte"
test_image = "MNIST/t10k-images-idx3-ubyte"
train_label = "MNIST/train-labels-idx1-ubyte"
test_label = "MNIST/t10k-labels-idx1-ubyte"
# 读取数据
train_x = read_image(train_image)  # train_image
test_x = read_image(test_image)  # test_image
train_y = read_label(train_label)  # train_label
test_y = read_label(test_label)  # test_label

##save_train_images = "SAVE/"
##for i in range(3):
##    path = save_train_images+str(i)+'.txt'
##    #打印文件路径
##    print(path)
##    #保存为txt格式
##    np.savetxt(path,train_x[i].reshape(28,28),fmt="%5d",delimiter=",##")
## print(train_x.shape)


# 设置运行参数

In [26]:
# 训练图片数
trainNum = 6000
# 测试图片数
testNum = 100
# 距离最小的k个图片
k = 10

train_Start = random.randint(0,60001-trainNum)
train_End = train_Start+trainNum
train_i = train_x[train_Start:train_End,:]
train_l = train_y[train_Start:train_End]

test_Start = random.randint(0,10001-testNum)
test_End = test_Start+testNum
test_i = test_x[test_Start:test_End,:]
test_l = test_y[test_Start:test_End]

# 运行

In [27]:
import datetime
start = datetime.datetime.now()

print('训练集图片数量：',train_i.shape[0],'区间:',train_Start,'~',train_End)
print('测试集图片数量：',test_i.shape[0],'区间:',test_Start,'~',test_End)

error=KNN_TEST(train_i,test_i,train_l,test_l,k)
for i in error:
    i+=test_Start
    print(i)

end = datetime.datetime.now()
print ('程序运行时间:',end-start)

训练集图片数量： 6000 区间: 44740 ~ 50740
测试集图片数量： 100 区间: 1886 ~ 1986
测试图片数量: 100
返回的结果是: 4, 真实结果是: 9
返回的结果是: 6, 真实结果是: 4
返回的结果是: 1, 真实结果是: 8
返回的结果是: 0, 真实结果是: 6
返回的结果是: 3, 真实结果是: 5
返回的结果是: 0, 真实结果是: 2

正确预测率: 0.940000

错误预测率: 0.060000

错误预测的图片数量: 6
1901
1938
1968
1969
1970
1984
程序运行时间: 0:00:04.347431


0